<a href="https://colab.research.google.com/github/MichelNivard/Boulder_hail/blob/main/01_common_variant_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Institute for Behavioral Genetics International Statistical Genetics 2023 Workshop 

## Common Variant Analysis of Sequencing Data with Hail

In this practical, we will learn how to:

1) Use simple python code and Jupyter notebooks.

2) Use Hail to import a VCF and run basic queries over sequencing data.

3) Use Hail to perform basic quality control on sequencing data.

4) Use Hail to run a basic genome-wide association study on common variants in sequencing data.


# Introduction


### It doesn't all need to "stick" today

This practical contains a lot of new material, and the goal of this workbook is not for you to be able to reproduce from memory all the various capabilities demonstrated here. Instead, the goal is for you to get a sense for the kind of analysis tasks that sequencing data requires, and to gain some exposure to what these analyses look like in Hail. 

There is no one-size-fits-all sequencing analysis pipeline, because each sequencing dataset will have unique properties that need to be understood and accounted for in QC and analysis. Hail can empower you to interrogate sequencing data, but it cannot give you all the questions to ask!

Some of the questions and exercises in this notebook might seem unrelated to the specific task of analyzing sequencing data, but that is intentional -- Hail is a computational tool that hopes to help you indulge your scientific curiosity, and asking and answering a variety of questions about many aspects of your data is the best way to learn *how to Hail*.

We don't expect you to be able to run a full GWAS on your own data in Hail tomorrow. If this is something you want to do, there are **lots more** resources available -- documentation, cookbooks, tutorials, and most importantly, the Hail community on the [forum](https://discuss.hail.is) and [zulip chatroom](https://hail.zulipchat.com).

### We encourage you to play

Hail is a highly expressive library with lots of functionality -- you'll see just a small fraction of it today. Throughout this notebook and especially in the denoted **exercises**, we encourage you to experiment with the code being run to see what happens! Sometimes it will be an error, but sometimes you will encounter new pieces of functionality. If you're curious about how to use Hail to ask slightly different questions than the code or exercises here, please ask the faculty! We are eager to help.

### Interactive analysis on the cloud

Part of what we think is so exciting about Hail is that Hail development has coincided with other technological shifts in the data science community.

Five years ago, most computational biologists analyzed sequencing data using command-line tools, and took advantage of research compute clusters by explicitly using scheduling frameworks like LSF or Sun Grid Engine. These clusters are powerful resources, but it requires a great deal of extra thought and effort to manage pipelines running on them.

Today, most Hail users run Hail from within interactive Python notebooks (like this one!) backed by thousands of cores on public compute clouds like [Google Cloud](https://cloud.google.com/), [Amazon Web Services](https://aws.amazon.com/), or [Microsoft Azure](https://azure.microsoft.com/). You don't need to share a cluster with hundreds of other scientists, and you only need to pay for the resources that you use.

You won't get hands-on experience with this kind of usage today, but there are lots of resources to help you get started if you're interested in that. Please stay in touch with us after the workshop ends!

# 1. Using Jupyter

The notebook software that you are using right now is called [Jupyter](https://jupyter.org/), which came from a combination of the languages **Ju**lia, **Pyt**hon, and **R**.

**Learning objectives**

 - be comfortable running, editing, adding, and deleting code cells.
 - learn techniques for unblocking yourself if Jupyter acts up.

### Running cells
Evaluate cells using SHIFT + ENTER. Select the next cell and run it. If you prefer clicking, you can select the cell and click the "Run" button in the toolbar above.

In [ ]:
print('Hello, world')

Hello, world


### Modes

Jupyter has two modes, a **navigation mode** and an **editor mode**.

#### Navigation mode:

 - <font color="blue"><strong>BLUE</strong></font> cell borders
 - `UP` / `DOWN` move between cells
 - `ENTER` while a cell is selected will move to **editing mode**.
 - Many letters are keyboard shortcuts! This is a common trap.
 
#### Editor mode:

 - <font color="green"><strong>GREEN</strong></font> cell borders
 - `UP` / `DOWN`/ move within cells before moving between cells.
 - `ESC` will return to **navigation mode**.
 - `SHIFT + ENTER` will evaluate a cell and return to **navigation mode**.
 
Try editing this markdown cell by double clicking, then re-rendering it by "running" the cell.

### Cell types

There are several types of cells in Jupyter notebooks. The two you will see in this notebook are **Markdown** (text) and **Code**.

In [ ]:
# This is a code cell
my_variable = 5

**This is a markdown cell**, so even if something looks like code (as below), it won't get executed!

my_variable += 1

### Shell commands

It is possible to call command-line utilities from Jupyter by prefixing a line with a `!`. For instance, we can print the current directory:

In [ ]:
! pwd

/home/timp/2023_IBG_HAIL/2021_IBG_Hail


### Tips and tricks

Keyboard shortcuts:

 - `SHIFT + ENTER` to evaluate a cell
 - `ESC` to return to navigation mode
 - `y` to turn a markdown cell into code
 - `m` to turn a code cell into markdown
 - `a` to add a new cell **above** the currently selected cell
 - `b` to add a new cell **below** the currently selected cell
 - `d, d` (repeated) to delete the currently selected cell
 - `TAB` to activate code completion
 
To try this out, create a new cell below this one using `b`, and print `my_variable` by starting with `print(my` and pressing `TAB`!

## <strong style="color: red;">Resetting Jupyter if you get stuck</strong>

If at any point during this practical, you are unable to successfully run cells, it is possible that your Python interpreter is in a bad state due to cells being run in an incorrect order. If this happens, you can recover a working session by doing the following:

1. Navigate to the "Kernel" menu at the top, and select "Restart and clear output".

2. Select the cell you were working on, then select "Run all above" from the "Cell" menu at the top.

3. If the problem persists, reach out to the faculty for help!

# 2. Import and initialize Hail

In addition to Hail, we import a few methods from the Hail plotting library. We'll see examples soon!

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 6G pyspark-shell'
import hail as hl
hl.init()

from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

23/03/10 09:37:38 WARN Utils: Your hostname, ip-10-0-201-172 resolves to a loopback address: 127.0.1.1; using 10.0.201.172 instead (on interface ens5)
23/03/10 09:37:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/10 09:37:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/10 09:37:39 WARN Hail: This Hail JAR was compiled for Spark 3.3.0, running with Spark 3.3.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.3.2
SparkUI available at http://10.0.201.172:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b123465fc0bf
LOGGING: writing to /home/timp/2023_IBG_HAIL/2021_IBG_Hail/hail-20230310-0937-0.2.109-b123465fc0bf.log


This notebook works on a small (~16MB) downsampled chunk of the publically available Human Genome Diversity Project (HGDP) dataset. HGDP is a super-set of the well-known [1000 genomes](https://www.internationalgenome.org/) dataset, with a broader group of represented populations.

We can see the files used using `ls` below:

In [ ]:
! ls -lh resources/

total 19M
-rwxr-xr-x 1 timp workshop  15K Mar 10 07:52 HGDP_sample_data.tsv
-rwxr-xr-x 1 timp workshop 2.5M Mar 10 07:52 ensembl_gene_annotations.txt
drwxr-xr-x 5 timp workshop 6.0K Mar 10 07:52 genes.ht
drwxr-xr-x 7 timp workshop 6.0K Mar 10 07:52 hgdp-tgp-rare-variants.mt
drwxr-xr-x 8 timp workshop 6.0K Mar 10 08:59 hgdp.mt
-rwxr-xr-x 1 timp workshop 382K Mar 10 07:52 hgdp_gene_annotations.tsv
-rwxr-xr-x 1 timp workshop  16M Mar 10 07:52 hgdp_subset.vcf.bgz
drwxr-xr-x 6 timp workshop 6.0K Mar 10 09:01 pca_scores.ht
drwxr-xr-x 5 timp workshop 6.0K Mar 10 07:52 rare-variant-phenotypes.ht


# 3. Explore genetic data with Hail

#### Learning Objectives:

- To be comfortable exploring Hail data structures, especially the `MatrixTable`.
- To understand categories of functionality for performing QC.

### Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail has an [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) function that reads this file to a Hail `MatrixTable`, which is a general-purpose data structure that is often used to represent a matrix of genetic data.

Why not work directly on the VCF? While VCF is a text format that is easy for humans to read, it is inefficient to process on a computer. 

The first thing we do is import (`import_vcf`) and convert the `VCF` file into a Hail native file format. This is done by using the `write` method below. Any queries that follow will now run much more quickly.

In [ ]:
hl.import_vcf('resources/hgdp_subset.vcf.bgz', min_partitions=4, reference_genome='GRCh38')\
.write('resources/hgdp.mt', overwrite=True)

### HGDP as a Hail `MatrixTable`

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [ ]:
mt = hl.read_matrix_table('resources/hgdp.mt')

### What is a `MatrixTable`?

Let's explore it!

You can see:
 - **numeric** types:
     - integers (`int32`, `int64`), e.g. `5`
     - floating point numbers (`float32`, `float64`), e.g. `5.5` or `3e-8`
 - **strings** (`str`), e.g. `"Foo"`
 - **boolean** values  (`bool`) e.g. `True`
 - **collections**:
     - arrays (`array`), e.g. `[1,1,2,3]`
     - sets (`set`), e.g. `{1,3}`
     - dictionaries (`dict`), e.g. `{'Foo': 5, 'Bar': 10}`
 - **genetic data types**:
     - loci (`locus`), e.g. `[GRCh37] 1:10000` or `[GRCh38] chr1:10024`
     - genotype calls (`call`), e.g. `0/2` or `1|0`

In [ ]:
mt.describe(widget=True)

### Exercise 1
Take a few moments to explore the interactive representation of the matrix table above.

* Where is the variant information (`locus` and `alleles`)? 
* Where is the sample identifier (`s`)?
* Where is the genotype quality `GQ`?

### `show`

Hail has a variety of functionality to help you quickly interrogate a dataset. The `show()` method prints the first few values of any field, and even prints in pretty HTML output in a Jupyter notebook! 

In [ ]:
mt.s.show()

""
s
str
"""LP6005441-DNA_F08"""
"""LP6005441-DNA_C05"""
"""HGDP00961"""
"""HGDP00804"""
"""HGDP00926"""
"""HGDP00716"""
"""HGDP01269"""
"""HGDP00241"""


It is also possible to show() the matrix table itself, which prints a portion of the top-left corner of the variant-by-sample matrix:

In [ ]:
# show() works fine with no arguments, but can print too little data by default on small screens!
mt.show(n_cols=3)

+---------------+------------+------------------------+------------------------+
| locus         | alleles    | 'LP6005441-DNA_F08'.GT | 'LP6005441-DNA_F08'.DP |
+---------------+------------+------------------------+------------------------+
| locus<GRCh38> | array<str> | call                   |                  int32 |
+---------------+------------+------------------------+------------------------+
| chr1:17379    | ["G","A"]  | 0/0                    |                     11 |
| chr1:95068    | ["G","A"]  | 0/0                    |                     15 |
| chr1:111735   | ["C","A"]  | 0/0                    |                     14 |
| chr1:134610   | ["G","A"]  | 0/0                    |                      8 |
| chr1:414783   | ["T","C"]  | NA                     |                     NA |
| chr1:1130877  | ["C","G"]  | 0/0                    |                     24 |
| chr1:1226707  | ["C","G"]  | 0/0                    |                     26 |
| chr1:1491494  | ["G","A"]  | 0/0                    |                     27 |
| chr1:1618118  | ["G","A"]  | 0/0                    |                     27 |
| chr1:2078529  | ["G","A"]  | 0/0                    |                     30 |
+---------------+------------+------------------------+------------------------+

+------------------------+------------------------+------------------------+
| 'LP6005441-DNA_F08'.GQ | 'LP6005441-DNA_F08'.AD | 'LP6005441-DNA_F08'.PL |
+------------------------+------------------------+------------------------+
|                  int32 | array<int32>           | array<int32>           |
+------------------------+------------------------+------------------------+
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     NA | NA                     | NA                     |
|                     40 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     40 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     50 | NA                     | NA                     |
+------------------------+------------------------+------------------------+

+------------------------+------------------------+------------------------+
| 'LP6005441-DNA_C05'.GT | 'LP6005441-DNA_C05'.DP | 'LP6005441-DNA_C05'.GQ |
+------------------------+------------------------+------------------------+
| call                   |                  int32 |                  int32 |
+------------------------+------------------------+------------------------+
| 0/0                    |                     15 |                     30 |
| 0/0                    |                     27 |                     30 |
| 0/0                    |                     15 |                     20 |
| NA                     |                     NA |                     NA |
| 0/0                    |                      9 |                     20 |
| 0/0                    |                     27 |                     30 |
| 0/0                    |                     25 |                     20 |
| 0/0                    |                     30 |                     20 |
| 0/0                    |                     30 |                     40 |
| 0/0                    |                     37 |                     60 |
+------------------------+------------------------+------------------------+

+------------------------+------------------------+----------------+
| 'LP6005441-DNA_C05'.AD | 'LP6005441-DNA_C05'.PL | 'HGDP00961'.GT |
+------------------------+------------------------+----------------+
| array<int32>           | arr

The above output is visually noisy because the matrix table has as lot of information in it. `show`ing just the called genotype (`GT`) is a bit more friendly.

The printed representation of GT is explained below, where `a` is the reference allele and `A` is the alternate allele:

`0/0` : homozygous reference or `aa`

`0/1` : heterozygous or `Aa`

`1/1` : homozygous alternate or `AA` 


In [ ]:
mt.GT.show()

,,,,,
,,'LP6005441-DNA_F08','LP6005441-DNA_C05','HGDP00961','HGDP00804'
locus,alleles,GT,GT,GT,GT
locus<GRCh38>,array<str>,call,call,call,call
chr1:17379,"[""G"",""A""]",0/0,0/0,0/0,0/0
chr1:95068,"[""G"",""A""]",0/0,0/0,NA,0/0
chr1:111735,"[""C"",""A""]",0/0,0/0,0/1,0/0
chr1:134610,"[""G"",""A""]",0/0,NA,NA,0/0
chr1:414783,"[""T"",""C""]",NA,0/0,NA,NA
chr1:1130877,"[""C"",""G""]",0/0,0/0,0/0,0/0
chr1:1226707,"[""C"",""G""]",0/0,0/0,0/0,0/0


### Exercise 2

There is a fourth value seen above, other than `0/0`, `0/1`, `1/1`. What is it?

### `summarize`
`summarize` Prints (potentially) useful information about any field or object:

`DP` is the read depth (number of short reads spanning a position for a given sample). Let's summarize all values of DP:

In [ ]:
mt.DP.summarize()

Non-missing,3851673 (94.11%)
Missing,241199 (5.89%)
Minimum,0
Maximum,5057
Mean,33.02
Std Dev,30.20


`AD` is the array of allelic depth per allele at a called genotype. Note especially the missingness properties:

In [ ]:
mt.AD.summarize()

Non-missing,1164892 (28.46%)
Missing,2927980 (71.54%)
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,2329784 (100.00%)
Missing,0
Minimum,0
Maximum,1299
Mean,17.09
Std Dev,15.13


### Exercise 3

In the empty cell below, summarize some of the other fields on the matrix table. You can use the interactive widget above to find the names of some of the other fields.


### `count`

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [ ]:
mt.count()

(10441, 392)

The count above tells us that we have 10,441 variants and 392 samples. This is just a tiny slice of a real sequencing dataset. The largest sequencing datasets today comprise hundreds of thousands of samples and more than a billion variants.

## Hail has a large library of genetics functionality

Hail can be used to analyze any kind of data (Hail team members have used Hail to analyze household financial data, USA election polling data, and even to build a bot that posts real-time updates about the Euro 2020 tournament to Slack). However, Hail does not have *only* general-purpose analysis functionality. Hail has a large set of functionality built for genetics and genomics.

For example, `hl.summarize_variants` prints useful statistics about the variants in the dataset. These are not part of the generic `summarize()` function, which must support all kinds of data, not just variant data!

In [ ]:
hl.summarize_variants(mt)

Number of alleles,Count
2,10441
Allele type,Count
SNP,10441
Metric,Value
Transitions,6602
Transversions,3839
Ratio,1.72
Contig,Count
chr1,881
chr2,799


# 4. Annotation and quality control

## Integrate sample information

Our dataset currently only has sample IDs and genetic data. In order to run a toy GWAS, we need phenotype information.

We can find it in the following file:

In [ ]:
! head resources/HGDP_sample_data.tsv

sample_id	pop	continental_pop	sex_karyotype	sleep_duration	tea_intake_daily	general_happiness	screen_time_per_day
HG00107	gbr	nfe	XY	6	3	3.2895e+00	11
HG00114	gbr	nfe	XY	5	3	3.5099e+00	10
HG00121	gbr	nfe	XX	6	6	2.0851e+00	6
HG00127	gbr	nfe	XX	6	3	2.7580e+00	6
HG00132	gbr	nfe	XX	5	6	2.2454e+00	5
HG00149	gbr	nfe	XY	5	6	2.8159e+00	9
HG00177	fin	fin	XX	7	9	3.3661e+00	8
HG00190	fin	fin	XY	5	6	2.9159e+00	6
HG00233	gbr	nfe	XX	8	3	3.9002e+00	10


We can import it as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it `sd` for "sample data".

In [ ]:
sd = hl.import_table('resources/HGDP_sample_data.tsv',
                     key='sample_id',
                     impute=True)

The "key" argument tells Hail to use the `sample_id` field as the table key, which is used to find matching values in  joins. In a moment, we will be joining the `sd` table onto our matrix table so that we can use the sample data fields in our QC and analysis. It is also possible to specify a new key for an existing table using the `.key_by(...)` method.

The "impute" argument tells Hail to impute the data types of the fields on the table. What does this mean? It means that you can ask Hail to figure out what is the data type in each column field such as `str` (string or just characters), `bool` (boolean or just true and false), `float64` (float or numbers with decimals), or `int32` (integer or numbers without decimals/whole numbers). If you don't use the `impute` flag or specify types manually with the `types` argument, each field will be imported as a string.

While we can see the names and types of fields in the logging messages and in the `head` output above, we can also `show` this table:

In [ ]:
sd.show()

,,,,,,,
sample_id,pop,continental_pop,sex_karyotype,sleep_duration,tea_intake_daily,general_happiness,screen_time_per_day
str,str,str,str,int32,int32,float64,int32
"""HG00107""","""gbr""","""nfe""","""XY""",6,3,3.29e+00,11
"""HG00114""","""gbr""","""nfe""","""XY""",5,3,3.51e+00,10
"""HG00121""","""gbr""","""nfe""","""XX""",6,6,2.09e+00,6
"""HG00127""","""gbr""","""nfe""","""XX""",6,3,2.76e+00,6
"""HG00132""","""gbr""","""nfe""","""XX""",5,6,2.25e+00,5
"""HG00149""","""gbr""","""nfe""","""XY""",5,6,2.82e+00,9
"""HG00177""","""fin""","""fin""","""XX""",7,9,3.37e+00,8
"""HG00190""","""fin""","""fin""","""XY""",5,6,2.92e+00,6


And we can `summarize` each field in `sd`:

In [ ]:
sd.summarize()

Non-missing,392 (100.00%)
Missing,0
Min Size,7
Max Size,17
Mean Size,7.92
Sample Values,"['HG00107', 'HG00114', 'HG00121', 'HG00127', 'HG00132']"
Non-missing,392 (100.00%)
Missing,0
Min Size,2
Max Size,11
Mean Size,3.84


## Add sample data to our HGDP `MatrixTable`

Let's now merge our genetic data (`mt`) with our sample data (`sd`).

This is a join between the `sd` table and the columns of our matrix table. It just takes one line:

In [ ]:
mt = mt.annotate_cols(sample_data = sd[mt.s])

### What's going on here?

Understanding what's going on here is a bit more difficult. To understand, we need to understand a few pieces:

#### 1. `annotate` methods

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "sample_data". This field should be the values in our table `sd` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

Python uses square brackets to look up values in dictionaries:

    >>> d = {'foo': 5, 'bar': 10}
    
    >>> d['foo']
    'bar'

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sd` using the sample ID `s`.

Let's see how the matrix table has changed:

In [ ]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'sample_data': struct {
        pop: str, 
        continental_pop: str, 
        sex_karyotype: str, 
        sleep_duration: int32, 
        tea_intake_daily: int32, 
        general_happiness: float64, 
        screen_time_per_day: int32
    }
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        QUALapprox: int32, 
        SB: array<int32>, 
        MQ: float64, 
        MQRankSum: float64, 
        VarDP: int32, 
        AS_ReadPosRankSum: float64, 
        AS_pab_max: float64, 
        AS_QD: float64, 
        AS_MQ: float64, 
        QD: float64, 
        AS_MQRankSum: float64, 
        FS: float64, 
        AS_FS: float64, 
        ReadPosRankSum: float64, 
        AS_QUALapprox: int32, 
        AS_SB_TA

### Cheat sheets

More information about matrix tables and tables can be found in a graphical representation as Hail cheat sheets:

 - [MatrixTable](https://hail.is/docs/0.2/_static/cheatsheets/hail_matrix_tables_cheat_sheet.pdf)
 - [Table](https://hail.is/docs/0.2/_static/cheatsheets/hail_tables_cheat_sheet.pdf)

# 5. Sample QC

In past workshop sessions you have learned the important adage: _garbage in, garbage out_. Good QC takes time and thoughtfulness but is necessary for robust results. 

Here, we run through some simple sample qc steps, but **these steps are not a one-size-fits-all solution for QC on your own data!**

Hail has the function [hl.sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) to compute a list of useful statistics about samples from sequencing data. This function adds a new column field, `sample_qc`, with the computed statistics. Note that this doesn't actually remove samples for you -- those decisions are up to you. The `sample_qc` method gives you some data you can use as a starting point.

**Click the sample_qc link** above to see the documentation, which lists the computed fields and their descriptions.

In [ ]:
mt = hl.sample_qc(mt)

In [ ]:
mt.describe(widget=True)

Hail includes a plotting library built that makes it easy to visualize fields of Hail tables and matrix tables.

Let's visualize the pairwise distribution of `Mean DP` (Read Depth) and `Call Rate`.

Note that you can **hover over points with your mouse to see the sample IDs!**

In [ ]:
(ggplot(mt, aes(x=mt.sample_qc.dp_stats.mean, y=mt.sample_qc.call_rate)) 
 + geom_point(aes(color=mt.sample_data.sex_karyotype))
 + xlab("Mean DP")
 + ylab("Call Rate"))


## Filter columns using generated QC statistics

Before filtering samples, we should compute a raw sample count:

In [ ]:
mt.count_cols()

392

`filter_cols` removes entire columns from the matrix table. Here, we keep columns (samples) where the `call_rate` is over 92%:

In [ ]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.92)

We can compute a final sample count:

In [ ]:
mt.count_cols()

385

How many samples did not meet your QC criteria?

# 6. Variant QC

Now that we have successfully gone through basic sample QC using the function `sample_qc` and general-purpose filtering methods, let's do variant QC.


Hail has the function [hl.variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) to compute a list of useful statistics about **variants** from sequencing data.

Once again, **Click the link** above to see the documentation!

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt.describe(widget=True)

Find the `variant_qc` output!

We can `show()` the computed information:

In [ ]:
mt.variant_qc.show()

+---------------+------------+--------------------------+
| locus         | alleles    | variant_qc.dp_stats.mean |
+---------------+------------+--------------------------+
| locus<GRCh38> | array<str> |                  float64 |
+---------------+------------+--------------------------+
| chr1:17379    | ["G","A"]  |                 4.95e+01 |
| chr1:95068    | ["G","A"]  |                 2.28e+01 |
| chr1:111735   | ["C","A"]  |                 1.42e+01 |
| chr1:134610   | ["G","A"]  |                 1.36e+01 |
| chr1:414783   | ["T","C"]  |                 6.27e+00 |
| chr1:1130877  | ["C","G"]  |                 3.75e+01 |
| chr1:1226707  | ["C","G"]  |                 3.61e+01 |
| chr1:1491494  | ["G","A"]  |                 3.41e+01 |
| chr1:1618118  | ["G","A"]  |                 3.63e+01 |
| chr1:2078529  | ["G","A"]  |                 3.41e+01 |
+---------------+------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  2.17e+01 |                7.00e+00 |
|                  1.19e+01 |                4.00e+00 |
|                  6.02e+00 |                2.00e+00 |
|                  5.46e+00 |                2.00e+00 |
|                  2.65e+00 |                1.00e+00 |
|                  7.77e+00 |                1.90e+01 |
|                  6.28e+00 |                2.30e+01 |
|                  5.39e+00 |                1.90e+01 |
|                  6.23e+00 |                2.10e+01 |
|                  4.28e+00 |                2.20e+01 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                1.41e+02 |                 6.44e+01 |
|                1.06e+02 |                 4.36e+01 |
|                2.80e+01 |                 2.73e+01 |
|                2.80e+01 |                 2.28e+01 |
|                1.40e+01 |                 1.50e+01 |
|                6.50e+01 |                 5.50e+01 |
|                7.70e+01 |                 5.26e+01 |
|                7.30e+01 |                 3.97e+01 |
|                7.60e+01 |                 6.50e+01 |
|                5.80e+01 |                 4.13e+01 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  2.84e+01 |                2.00e+01 |
|                  3.19e+01 |                1.20e+01 |
|                  1.87e+01 |                1.00e+00 |
|                  6.94e+00 |                6.00e+00 |
|                  4.73e+00 |                3.00e+00 |
|                  2.82e+01 |                2.00e+01 |
|                  2.16e+01 |                2.00e+01 |
|                  1.92e+01 |                2.00e+01 |
|                  2.67e+01 |                2.00e+01 |
|                  1.60e+01 |                2.00e+01 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 float64 | array<int32>  | array<float64>      |
+-------------------------+---------------+---------------------+
|                9.90e+01 | [752,10]      | [9.87e-01,1.31e-02] |


Metrics like `call_rate` are important for QC. Let's plot the cumulative density function of call rate per variant:

In [ ]:
ggplot(mt, aes(x=mt.variant_qc.call_rate)) + geom_density() + xlab('Variant Call Rate') + ylab('Density')

Before filtering variants, we should compute a raw variant count:

In [ ]:
# pre-qc variant count
mt.count_rows()

10441

`filter_rows` removes entire rows of the matrix table. Here, we keep rows where the `call_rate` is over 95%:

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

### Hardy-Weinberg filter

In past sessions, you have seen filters on Hardy-Weinberg equilibrium using tools like PLINK.

Hail can do the same. First, let's plot the log-scaled HWE p-values per variant:

In [ ]:
(ggplot(mt, aes(x=-hl.log10(mt.variant_qc.p_value_hwe)))
 + geom_histogram()
 + xlab("-log10 pHWE")
 + ylab("density"))

There are some **extremely** bad variants here, with p-values smaller than 1e-100. Let's look at some of these variants:

In [ ]:
rows = mt.rows()
rows.order_by(rows.variant_qc.p_value_hwe).show()

+----------------+------------+----------------+-----------+
| locus          | alleles    | rsid           |      qual |
+----------------+------------+----------------+-----------+
| locus<GRCh38>  | array<str> | str            |   float64 |
+----------------+------------+----------------+-----------+
| chr1:125165544 | ["G","T"]  | NA             | -1.00e+01 |
| chr2:94565722  | ["T","C"]  | "rs1273701572" | -1.00e+01 |
| chr2:113591601 | ["G","A"]  | "rs6419550"    | -1.00e+01 |
| chr3:77780883  | ["A","T"]  | "rs75079699"   | -1.00e+01 |
| chr5:46433521  | ["G","C"]  | NA             | -1.00e+01 |
| chr5:170833888 | ["C","T"]  | "rs79126011"   | -1.00e+01 |
| chr9:63771196  | ["C","T"]  | "rs4310313"    | -1.00e+01 |
| chr10:46546502 | ["G","A"]  | "rs3127692"    | -1.00e+01 |
| chr17:21291540 | ["G","A"]  | "rs72840058"   | -1.00e+01 |
| chr17:21302646 | ["A","G"]  | "rs62057725"   | -1.00e+01 |
+----------------+------------+----------------+-----------+

+---------------------+-----------------+-----------------------------------+
| filters             | info.QUALapprox | info.SB                           |
+---------------------+-----------------+-----------------------------------+
| set<str>            |           int32 | array<int32>                      |
+---------------------+-----------------+-----------------------------------+
| {"InbreedingCoeff"} |       332680306 | [2530347,2658319,4674032,5445216] |
| {"InbreedingCoeff"} |       133145705 | [2744392,2448688,2632227,938989]  |
| {"InbreedingCoeff"} |       124592291 | [3447869,3213349,2778556,2338468] |
| {"InbreedingCoeff"} |       179360814 | [2431382,2598153,2418400,2106424] |
| {"InbreedingCoeff"} |       115906887 | [2362879,2344419,2204239,1144094] |
| {"InbreedingCoeff"} |       157871544 | [1038041,2758564,1388629,2553316] |
| {"InbreedingCoeff"} |       113903241 | [9577761,6255003,2256377,1811696] |
| {"InbreedingCoeff"} |       152605413 | [2875916,2474502,3025177,2601488] |
| {"InbreedingCoeff"} |       204637824 | [2818670,2658428,2716858,2427104] |
| {"InbreedingCoeff"} |       166814776 | [2745828,2529213,2725172,2431983] |
+---------------------+-----------------+-----------------------------------+

+----------+----------------+------------+------------------------+
|  info.MQ | info.MQRankSum | info.VarDP | info.AS_ReadPosRankSum |
+----------+----------------+------------+------------------------+
|  float64 |        float64 |      int32 |                float64 |
+----------+----------------+------------+------------------------+
| 5.92e+01 |      -1.72e-01 |   15307914 |               5.29e-01 |
| 5.38e+01 |      -5.59e+00 |    8764281 |               1.06e+00 |
| 4.69e+01 |      -2.27e+00 |   11778240 |               9.33e-01 |
| 6.00e+01 |       0.00e+00 |    9554359 |              -1.42e-01 |
| 5.32e+01 |      -2.23e+00 |    8055631 |               9.49e-01 |
| 5.99e+01 |      -5.50e-02 |    7738550 |              -7.80e-02 |
| 4.26e+01 |      -6.49e-01 |   19900836 |               5.55e-01 |
| 6.00e+01 |      -4.00e-03 |   10977076 |               4.96e-01 |
| 6.00e+01 |      -1.40e-02 |   10621059 |               3.76e-01 |
| 6.00e+01 |      -8.00e-03 |   10432195 |               7.45e-01 |
+----------+----------------+------------+------------------------+

+-----------------+------------+------------+----------+-------------------+
| info.AS_pab_max | info.AS_QD | info.AS_MQ |  info.QD | info.AS_MQRankSum |
+-----------------+------------+------------+----------+-------------------+
|         float64 |    float64 |    float64 |  float64 |           float64 |
+-----------------+------------+------------+----------+-------------------+
|        1.00e+00 |   2.17e+01 |   5.92e+01 | 2.17e+01 |         -1.77e-01 |
|        1.00e+00 |   1.52e+01 |   5.38e+01 | 1.52e+01 |         -5.61e+00 |
|        1.00e+00 |   1.06e+01 |   4.69e+01 | 1.06e+01 |         -2.29e+00 |
|        1.00e+00 |   1.88e+01 |   6.00e+01 | 1.88e+01 |         -6.0

### Exercise 4

There's a lot of information in the above output. Take a moment to look through, and remember, these are **bad-quality variants**. Why do these variants had such low HWE p-values? *Hint: scroll all the way to the right to the variant_qc output*.

### The VCF `FILTERS` field

You're not entirely on your own for variant QC -- most variant calling software generates its own filtering annotations that are present in the `FILTERS` field of the VCF. Lines of the VCF might have reasons to be filtered, or might be `PASS`. Above, almost every one of these bad variants has the `"InbreedingCoeff"` filter listed in its `mt.filters` field!

We can remove all of these pre-filtered variants by keeping only variants which have no filters applied.

In [ ]:
mt = mt.filter_rows(hl.len(mt.filters) == 0)

We can then compute the final sample and variant count:

In [ ]:
mt.count()

(8538, 385)

How many variants did you lose from your variant QC? 

# 7. Association Testing and PCA

The goal of a GWAS is to find associations between genotypes and a trait of interest.

First, we filter to common variants (those with a minor allele frequency over 1%). GWAS is not well-powered to detect signal from extremely rare variants, like those only observed in one individual.

Our filter below removes variants where the minimum value of AF is below 1%. The AF array computed by `hl.variant_qc` contains one value for each allele, **including the reference**, and sums to 1.0.

A variant with 5% MAF might have AF values `[0.95, 0.05]` or `[0.05, 0.95]`. A variant with 0.1% MAF might have AF values `[0.999, 0.001]` or `[0.001, 0.999]`. This filter ensures that we account for both cases.

In [ ]:
base = mt
mt = base.filter_rows(hl.min(base.variant_qc.AF) > 0.01)

In [ ]:
mt.count()

(5862, 385)

How many variants did you lose from your common variant filter? 

Remember that in a GWAS we're running independent association tests on each variant.

In Hail, the method we use is [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows). Why isn't this called `hl.gwas`? Modularity! There are applications for this statistical method other than genome wide association studies.

We will start by using the `sleep_duration` phenotype. At the end, you will have a chance to try the others (you can do that by editing the string below to one of the other phenotypes and rerunning the rest of the notebook). The other phenotypes are:

 - 'tea_intake_daily'
 - 'general_happiness'
 - 'screen_time_per_day'

In [ ]:
phenotype = 'sleep_duration'

In [ ]:
gwas = hl.linear_regression_rows(y=mt.sample_data[phenotype], 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0]  # intercept term
                                )

The resulting output from the line above is a `table`. Let's look at what the table looks like.

Out of all of the fields, we would recommend focusing your understanding of the p-value and beta effect when determining if you have a GWAS signal.

**However**, keep in mind that the results thus far may need your model to be adjusted.

In [ ]:
gwas.show()

,,,,,,,,
locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh38>,array<str>,int32,float64,float64,float64,float64,float64,float64
chr1:17379,"[""G"",""A""]",385,1.01e+01,5.18e+01,-9.35e-01,4.64e-01,-2.02e+00,4.45e-02
chr1:1130877,"[""C"",""G""]",385,1.20e+01,7.20e+01,-2.41e-02,4.27e-01,-5.65e-02,9.55e-01
chr1:1491494,"[""G"",""A""]",385,9.00e+00,5.30e+01,-1.38e-01,4.91e-01,-2.81e-01,7.79e-01
chr1:1618118,"[""G"",""A""]",385,1.29e+02,7.67e+02,-9.27e-02,1.28e-01,-7.26e-01,4.68e-01
chr1:3008858,"[""A"",""G""]",385,2.33e+02,1.44e+03,1.52e-01,1.01e-01,1.51e+00,1.32e-01
chr1:3254545,"[""T"",""C""]",385,2.83e+02,1.76e+03,2.53e-01,9.75e-02,2.59e+00,9.83e-03
chr1:3779436,"[""T"",""C""]",385,3.72e+02,2.17e+03,-2.60e-01,8.92e-02,-2.91e+00,3.78e-03
chr1:4156721,"[""C"",""T""]",385,2.31e+01,1.46e+02,2.76e-01,2.99e-01,9.23e-01,3.57e-01


# 8. Visualization

Let’s visualize our association test results from the linear regression. We can do so by creating 2 common plots: a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) and a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

We'll start with the Manhattan plot:

In [ ]:
manhattan_plot(gwas.p_value)

### Manhattan plot interpretation

The dashed red line is the line for genome-wide significance with a typical Bonferroni correction: 5e-8.

We have several points above the line -- mouse over to see the loci they refer to. This means we're ready to publish our sleep GWAS in *Nature*, right?

...right?

### Q-Q plot
The plot that accompanies a Manhattan plot is the Q-Q (quantile-quantile) plot.

In [ ]:
qq_plot(gwas.p_value)

The Q-Q plot can clearly indicate widespread inflation or deflation of p-values. Is either of those properties present here?

### Exercise 5

**Is this GWAS well controlled? Discuss with your group.**

Wikipedia has a good description of [genomic control estimation](https://en.wikipedia.org/wiki/Genomic_control) (lambda GC) to read later.

# 9. Confounded! PCA to the rescue.

If you've done a GWAS before, you've probably included a few other covariates that might be confounders -- age, sex, and principal components.

Principal components are a measure of genetic ancestry, and can be used to control for [population stratification](https://en.wikipedia.org/wiki/Population_stratification).

Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA typically works best on normalized data (e.g. mean centered). Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True)

In [ ]:
pca_scores = pca_scores.checkpoint('resources/pca_scores.ht', overwrite=True)

The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

### Joining computed PCs onto our matrix table

Using the same syntax as we used to join the sample data table `sd` above, we can join the computed scores onto our matrix table:

In [ ]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

## Visualize principal components

Let's make plots!

In [ ]:
(ggplot(mt,
        aes(x=mt.pca.scores[0],
            y=mt.pca.scores[1],
            color=mt.sample_data.continental_pop.upper(),
            tooltip=(mt.sample_data.pop, mt.sample_data.continental_pop.upper())))
 + geom_point()
 + xlab('PC1')
 + ylab('PC2'))

In [ ]:
(ggplot(mt,
        aes(x=mt.pca.scores[2],
            y=mt.pca.scores[3],
            color=mt.sample_data.continental_pop.upper(),
            tooltip=(mt.sample_data.pop, mt.sample_data.continental_pop.upper())))
 + geom_point()
 + xlab('PC3')
 + ylab('PC4'))

# 10. Control confounders and run another GWAS

Now that we have computed principal components and saved it into our `mt`, let’s run another GWAS that includes PCs as covariates to account for confounding due to population stratification.


In [ ]:
gwas2 = hl.linear_regression_rows(
    y=mt.sample_data[phenotype],
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pca.scores[0], mt.pca.scores[1], mt.pca.scores[2]])

In [ ]:
qq_plot(gwas2.p_value)

The above Q-Q plot indicates a much better-controlled GWAS. Let's try the Manhattan plot:

In [ ]:
manhattan_plot(gwas2.p_value)

### Are there any significant hits? Do you trust these results? Why or why not?

Discuss with your group - roughly how many samples would you need to discover rare variant signal? polygenic signal?

### Exercise 6

Change the "gwas2" cell to experiment with how many principal components are needed as covariates to properly control this GWAS. How many are needed here in our tiny simulated example? How many are needed in a typical GWAS?